In [41]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw 
from netgen.occ import *
from netgen.csg import *
from scipy.optimize import minimize_scalar
from netgen.geom2d import SplineGeometry

import sys
from IPython.display import clear_output
sys.path.insert(0,"../")
sys.path.insert(0,"../../")
import problems
import numpy as np
import params
import pickle
from time import time

In [42]:


problem = problems.problem2

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 0.4

ord = 1

N = params.N
KBTV = params.KBTV
form = "Functional" 


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
# def mesher(geom, h):
#     if ".stp" in geom:
#         geo = OCCGeometry(geom)
#     else:
#         geo = pickle.load(open(geom, "rb"))

#     mesh = Mesh(geo.GenerateMesh(maxh=h))
#     return mesh
# mesh = mesher(geom, h)  
w,l= 3,15
n = 5
rect = SplineGeometry()
pnts = [(0,0), (l,0), (l,w), (0,w)]
p1,p2,p3,p4 = [rect.AppendPoint(*pnt,hpref=1) for pnt in pnts]
curves = [[["line",p1,p2],"bottom"],
        [["line",p2,p3],"right"],
        [["line",p3,p4],"top"],
        [["line",p4,p1],"left"]]
rect.Append(curves[0][0], bc = curves[0][1], leftdomain=1, rightdomain=0)
rect.Append(curves[1][0], bc = curves[1][1], leftdomain=1, rightdomain=0)
rect.Append(curves[2][0], bc = curves[2][1], leftdomain=1, rightdomain=0)
rect.Append(curves[3][0], bc = curves[3][1], leftdomain=1, rightdomain=0)


mesh = Mesh(rect.GenerateMesh(maxh=0.5))
def Gel_energy_functional(F):
    
    g = 9.81
    J = Det(F)
    rho = 1.23e-6
    H = (J - phi0)*log(1-phi0/J)  + phi0 * chi*(1-phi0/J)
    # min (G/2) * (3 lam**2) + KBTV * H(lam**3):  lam > 1
    Hf = lambda lam: (lam-phi0)*np.log(1-phi0/lam) + phi0*chi*(1-phi0/lam)

    fun = lambda lam: (G.Get()/2)*(3*lam**2) + KBTV*Hf(lam)
    res = minimize_scalar(fun, bounds=(1, 10), method='bounded')
    C = res.fun
    return 0.5 * (G) * Trace(F.trans*F ) + H*KBTV #+ rho*g*u[1] - C

"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fes = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fes = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])


u = fes.TrialFunction()
v = fes.TestFunction()
BF = BilinearForm(fes)


eps = 1e-6
# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.vec[:] = 0

obs = GridFunction(fes)

obs.Set(CF((0,-y)))

def Assemble_BF(BF,u,form):
    F = Id(2)+ Grad(u)
    if form == "Functional":
        BF += Variation(Gel_energy_functional(F).Compile()*dx)
        return BF





In [ ]:
max_iter_newton = 100
newton_damp = 0.3
softening_n = 30

tol_newton = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*25,softening_n))

BF = Assemble_BF(BF,u,"Functional")
for num, load in enumerate(gammas):
    Draw(gfu.components[1], mesh, "gfu")
    print("Softening step", num, "load", load)
    G.Set(load) #incremental softening
    if num == softening_n-1:
        tol_newton = 1e-8
    res = gfu.vec.CreateVector()
    w = gfu.vec.CreateVector()

    for iter in range(max_iter_newton):
        with TaskManager():
            BF.Apply(gfu.vec,res)
            BF.AssembleLinearization(gfu.vec)
            inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
            w.data = newton_damp * inv * res
            gfu.vec.data += -w
            
            gfu.components[1].Set(IfPos(gfu.components[1] + y , gfu.components[1], -y+0.0001))
            delt = InnerProduct(w,res)

        if abs(delt) < tol_newton:
            print("Tol achieved newton",delt )
            break


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Softening step 0 load 3.25


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Softening step 1 load 3.142413793103448


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

...

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Softening step 22 load 0.8831034482758621


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Softening step 23 load 0.7755172413793103

UMFPACK V5.7.4 (Feb 1, 2016): WARNING: matrix is singular



NgException: UmfpackInverse: Numeric factorization failed.

In [45]:
Draw(gfu.components[1]*1000, mesh, "gfu")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
Draw